In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split, RepeatedStratifiedKFold
from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.svm import SVC
from sklearn.utils import resample
import skopt

from xgboost import XGBClassifier
import lightgbm as lgb

import pickle

%matplotlib inline
%precision 4

import warnings
warnings.filterwarnings('ignore')


In [2]:
roi_list = ["temporalpole", "frontalpole", "bankssts", "superiortemporal", "middletemporal", 
               "precentral", "postcentral", "supramarginal", "superiorparietal", "precuneus", "cuneus", 
               "pericalcarine", "lingual", "superiorfrontal", "rostralanteriorcingulate", "caudalanteriorcingulate",
               "posteriorcingulate", "isthmuscingulate", "medialorbitofrontal", "inferiortemporal", "lateraloccipital",
               "inferiorparietal", "caudalmiddlefrontal", "rostralmiddlefrontal", "lateralorbitofrontal",
               "parsorbitalis", "parstriangularis", "parsopercularis", "insula", "transversetemporal", "entorhinal",
               "paracentral", "fusiform", "parahippocampal"]


prec_ukbb = []
rec_ukbb = []
acc_ukbb = []
balacc_ukbb = []
f1_ukbb = []
spec_ukbb = []

prec_pnc = []
rec_pnc = []
acc_pnc = []
balacc_pnc = []
f1_pnc = []
spec_pnc = []

prec_ppmi = []
rec_ppmi = []
acc_ppmi = []
balacc_ppmi = []
f1_ppmi = []
spec_ppmi = []

prec_adni = []
rec_adni = []
acc_adni = []
balacc_adni = []
f1_adni = []
spec_adni = []
    
df_metric_ukbb = pd.DataFrame()
df_metric_pnc = pd.DataFrame()
df_metric_ppmi = pd.DataFrame()
df_metric_adni = pd.DataFrame()

df_prediction_adni = pd.DataFrame()
df_prediction_ppmi = pd.DataFrame()
df_prediction_pnc = pd.DataFrame()
df_prediction_ukbb = pd.DataFrame()

df_metric_ukbb["ROI"] = roi_list
df_metric_pnc["ROI"] = roi_list
df_metric_ppmi["ROI"] = roi_list
df_metric_adni["ROI"] = roi_list


In [3]:
def train_evaluate(search_params):
    X_trn, X_valid, y_trn, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=1234)

    train_data = lgb.Dataset(X_trn, label=y_trn)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

    params = {'objective': 'binary',
              'metric': 'auc',
              'is_unbalance': True,
              **search_params}

    model = lgb.train(params, train_data,
                      num_boost_round=300,
                      early_stopping_rounds=30,
                      valid_sets=[valid_data],
                      valid_names=['valid'])

    score = model.best_score['valid']['auc']
    return score


In [4]:
for selected_roi in roi_list:
    #Read data
    df_ukbb = pd.read_csv('./UKBB_v5p3/merged.csv')
    df_pnc = pd.read_csv('./PNC/merged.csv')
    df_ppmi = pd.read_csv('./PPMI/merged.csv')
    test_adni = pd.read_csv('./ADNI/merged.csv')

    #Train dataset
    df_ukbb['score'] = 0
    df_ukbb.loc[df_ukbb[selected_roi].isin(['L', 'R', ' R', 'R/L', 'RL']), ['score']] = 1
    
    df_pnc['score'] = 0
    df_pnc.loc[df_pnc[selected_roi].isin(['L', 'R', ' R', 'R/L', 'RL']), ['score']] = 1

    df_ppmi['score'] = 0
    df_ppmi.loc[df_ppmi[selected_roi].isin(['L', 'R', ' R', 'R/L', 'RL']), ['score']] = 1

    #Test dataset
    test_adni['score'] = 0
    test_adni.loc[test_adni[selected_roi].isin(['L', 'R', ' R', 'R/L', 'RL']), ['score']] = 1

    #Concatenate train datasets
    df_ukbb = df_ukbb.drop(['NP_controls_2', 'AgeAtScan', 'site_id', 'NP_controls_1', 'CNS_controls_1', 'sex', 'YearsOfEducation', 'ISCED', 'CNS_controls_2', 'Race'], axis=1)
    df_ukbb=df_ukbb.rename(columns={"eid": "SubjID"})
    
    train_ukbb, test_ukbb = train_test_split(df_ukbb, test_size = 0.3, random_state = 1)
    train_pnc, test_pnc = train_test_split(df_pnc, test_size = 0.3, random_state = 1)
    train_ppmi, test_ppmi = train_test_split(df_ppmi, test_size = 0.3, random_state = 1)
    train = pd.concat([train_ukbb, train_pnc, train_ppmi])

    #Split data
    X_train = train.drop(['SubjID', 'score'], axis = 1)
    X_test_ukbb = test_ukbb.drop(["SubjID", "score"], axis=1) 
    X_test_pnc = test_pnc.drop(["SubjID", "score"], axis=1) 
    X_test_ppmi = test_ppmi.drop(["SubjID", "score"], axis=1) 
    X_test_adni = test_adni.drop(["SubjID", "score"], axis=1)
    
    y_train = train['score']
    y_test_ukbb = test_ukbb['score']
    y_test_pnc = test_pnc['score']
    y_test_ppmi = test_ppmi['score']
    y_test_adni = test_adni['score']
    
    #Save the test subject id's in prediction sheet
    df_prediction_ukbb["SubjID"] = test_ukbb["SubjID"]
    df_prediction_pnc["SubjID"] = test_pnc["SubjID"]
    df_prediction_ppmi["SubjID"] = test_ppmi["SubjID"]
    df_prediction_adni["SubjID"] = test_adni["SubjID"]

    #Selected columns in dataframe
    columns = list(pd.read_csv('./features.csv').Features)
    X_train = X_train[columns]
    X_test_ukbb = X_test_ukbb[columns]
    X_test_pnc = X_test_pnc[columns]
    X_test_ppmi = X_test_ppmi[columns]
    X_test_adni = X_test_adni[columns]

    #Impute
    X_train = SimpleImputer().fit(X_train).transform(X_train)
    X_test_ukbb = SimpleImputer().fit(X_test_ukbb).transform(X_test_ukbb)
    X_test_pnc = SimpleImputer().fit(X_test_pnc).transform(X_test_pnc)
    X_test_ppmi = SimpleImputer().fit(X_test_ppmi).transform(X_test_ppmi)
    X_test_adni = SimpleImputer().fit(X_test_adni).transform(X_test_adni)

    #Normalize
    X_train = normalize(X_train)
    X_test_ukbb = normalize(X_test_ukbb)
    X_test_pnc = normalize(X_test_pnc)
    X_test_ppmi = normalize(X_test_ppmi)
    X_test_adni = normalize(X_test_adni)

    #Scaling
    minmax = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    filename = './models/minimax.sav'
    pickle.dump(minmax, open(filename, 'wb'))
    
    X_train = minmax.transform(X_train)
    X_test_ukbb = minmax.transform(X_test_ukbb)
    X_test_pnc = minmax.transform(X_test_pnc)
    X_test_ppmi = minmax.transform(X_test_ppmi)
    X_test_adni = minmax.transform(X_test_adni)

    #Generate dataframe
    X_train = pd.DataFrame(X_train)
    X_train.columns = columns

    X_test_ukbb = pd.DataFrame(X_test_ukbb)
    X_test_ukbb.columns = columns

    X_test_pnc = pd.DataFrame(X_test_pnc)
    X_test_pnc.columns = columns

    X_test_ppmi = pd.DataFrame(X_test_ppmi)
    X_test_ppmi.columns = columns

    X_test_adni = pd.DataFrame(X_test_adni)
    X_test_adni.columns = columns

    # Search for best model parameters
    SEARCH_PARAMS = {'learning_rate': 0.1,
                 'max_depth': 1,
                 'num_leaves': 7,
                 'feature_fraction': 0.6, ##### feature_fraction = bagging_fraction
                 'subsample': 0.2}
    
    SPACE = [
        skopt.space.Real(0.0001, 0.5, name='learning_rate', prior='log-uniform'),
        skopt.space.Integer(10, 50, name='max_depth'),
        skopt.space.Integer(50, 150, name='num_leaves'),
        skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),
        skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')
    ]

    @skopt.utils.use_named_args(SPACE)
    def objective(**params):
        return -1.0 * train_evaluate(params)

    results = skopt.forest_minimize(objective, SPACE, n_calls=100, base_estimator='RF')
    best_params = results['x']

    #Light Gradient Boost Classifier with random forest as the base estimator
    clf = lgb.LGBMClassifier(boosting_type='rf', is_unbalance=True, verbose=4, objective='binary', 
                             importance_type='gain', n_estimators=100, 
                             learning_rate=best_params[0], 
                             max_depth=best_params[1], 
                             num_leaves=best_params[2], 
                             bagging_fraction = best_params[3],
                             subsample=best_params[4],
                             bagging_freq=1)
    
    filename = "./models/model_" + str(selected_roi) + ".sav"
    pickle.dump(clf, open(filename, 'wb'))

    #Training
    clf.fit(X_train, y_train)
    
    #Prediction on all test sets
    y_pred_ukbb_test = clf.predict(X_test_ukbb)
    y_pred_pnc_test = clf.predict(X_test_pnc)
    y_pred_ppmi_test = clf.predict(X_test_ppmi)
    y_pred_adni_test = clf.predict(X_test_adni)
    
    #Get confusion matrix
    cm_ukbb = confusion_matrix(y_test_ukbb, y_pred_ukbb_test)
    cm_pnc = confusion_matrix(y_test_pnc, y_pred_pnc_test)
    cm_ppmi = confusion_matrix(y_test_ppmi, y_pred_ppmi_test)
    cm_adni = confusion_matrix(y_test_adni, y_pred_adni_test)

    #Store Precision, recall, accuracy, f1 score, balanced accuracy
    prec_ukbb.append(precision_score(y_test_ukbb, y_pred_ukbb_test, average='weighted'))
    rec_ukbb.append(recall_score(y_test_ukbb, y_pred_ukbb_test, average='weighted'))
    acc_ukbb.append(accuracy_score(y_test_ukbb, y_pred_ukbb_test))
    balacc_ukbb.append(balanced_accuracy_score(y_test_ukbb, y_pred_ukbb_test))
    f1_ukbb.append(f1_score(y_test_ukbb, y_pred_ukbb_test, average='weighted'))
    if cm_ukbb.shape == (1,1):
        spec_ukbb.append("-")
    else:
        spec_ukbb.append(cm_ukbb[1,1]/(cm_ukbb[1,0]+cm_ukbb[1,1]))

    prec_pnc.append(precision_score(y_test_pnc, y_pred_pnc_test, average='weighted'))
    rec_pnc.append(recall_score(y_test_pnc, y_pred_pnc_test, average='weighted'))
    acc_pnc.append(accuracy_score(y_test_pnc, y_pred_pnc_test))
    balacc_pnc.append(balanced_accuracy_score(y_test_pnc, y_pred_pnc_test))
    f1_pnc.append(f1_score(y_test_pnc, y_pred_pnc_test, average='weighted'))
    if cm_pnc.shape == (1,1):
        spec_pnc.append("-")
    else:
        spec_pnc.append(cm_pnc[1,1]/(cm_pnc[1,0]+cm_pnc[1,1]))

    prec_ppmi.append(precision_score(y_test_ppmi, y_pred_ppmi_test, average='weighted'))
    rec_ppmi.append(recall_score(y_test_ppmi, y_pred_ppmi_test, average='weighted'))
    acc_ppmi.append(accuracy_score(y_test_ppmi, y_pred_ppmi_test))
    balacc_ppmi.append(balanced_accuracy_score(y_test_ppmi, y_pred_ppmi_test))
    f1_ppmi.append(f1_score(y_test_ppmi, y_pred_ppmi_test, average='weighted'))
    if cm_ppmi.shape == (1,1):
        spec_ppmi.append("-")
    else:
        spec_ppmi.append(cm_ppmi[1,1]/(cm_ppmi[1,0]+cm_ppmi[1,1]))

    prec_adni.append(precision_score(y_test_adni, y_pred_adni_test, average='weighted'))
    rec_adni.append(recall_score(y_test_adni, y_pred_adni_test, average='weighted'))
    acc_adni.append(accuracy_score(y_test_adni, y_pred_adni_test))
    balacc_adni.append(balanced_accuracy_score(y_test_adni, y_pred_adni_test))
    f1_adni.append(f1_score(y_test_adni, y_pred_adni_test, average='weighted'))
    if cm_adni.shape == (1,1):
        spec_adni.append("-")
    else:
        spec_adni.append(cm_adni[1,1]/(cm_adni[1,0]+cm_adni[1,1]))

    df_prediction_adni[str(selected_roi)] = y_pred_adni_test
    df_prediction_ukbb[str(selected_roi)] = y_pred_ukbb_test
    df_prediction_pnc[str(selected_roi)] = y_pred_pnc_test
    df_prediction_ppmi[str(selected_roi)] = y_pred_ppmi_test
    
    print(selected_roi + " done")


temporalpole done
frontalpole done
bankssts done
superiortemporal done
middletemporal done
precentral done
postcentral done
supramarginal done
superiorparietal done
precuneus done
cuneus done
pericalcarine done
lingual done
superiorfrontal done
rostralanteriorcingulate done
caudalanteriorcingulate done
posteriorcingulate done
isthmuscingulate done
medialorbitofrontal done
inferiortemporal done
lateraloccipital done
inferiorparietal done
caudalmiddlefrontal done
rostralmiddlefrontal done
lateralorbitofrontal done
parsorbitalis done
parstriangularis done
parsopercularis done
insula done
transversetemporal done
entorhinal done
paracentral done
fusiform done
parahippocampal done


In [5]:
#Create a dataframe for all the performance metrics
df_metric_ukbb["Precision"] = prec_ukbb
df_metric_ukbb["Recall"] = rec_ukbb
df_metric_ukbb["Accuracy"] = acc_ukbb
df_metric_ukbb["Balanced Accuracy"] = balacc_ukbb
df_metric_ukbb["F1"] = f1_ukbb
df_metric_ukbb["Specificity"] = spec_ukbb

df_metric_pnc["Precision"] = prec_pnc
df_metric_pnc["Recall"] = rec_pnc
df_metric_pnc["Accuracy"] = acc_pnc
df_metric_pnc["Balanced Accuracy"] = balacc_pnc
df_metric_pnc["F1"] = f1_pnc
df_metric_pnc["Specificity"] = spec_pnc

df_metric_ppmi["Precision"] = prec_ppmi
df_metric_ppmi["Recall"] = rec_ppmi
df_metric_ppmi["Accuracy"] = acc_ppmi
df_metric_ppmi["Balanced Accuracy"] = balacc_ppmi
df_metric_ppmi["F1"] = f1_ppmi
df_metric_ppmi["Specificity"] = spec_ppmi

df_metric_adni["Precision"] = prec_adni
df_metric_adni["Recall"] = rec_adni
df_metric_adni["Accuracy"] = acc_adni
df_metric_adni["Balanced Accuracy"] = balacc_adni
df_metric_adni["F1"] = f1_adni
df_metric_adni["Specificity"] = spec_adni


In [6]:
#Save the performance metrics
df_metric_ukbb.to_csv("./results/performance_metrics_ukbb_test.csv")
df_metric_ppmi.to_csv("./results/performance_metrics_ppmi_test.csv")
df_metric_pnc.to_csv("./results/performance_metrics_pnc_test.csv")
df_metric_adni.to_csv("./results/performance_metrics_adni_test.csv")

#Save the QC predictions
df_prediction_ukbb.to_csv("./results/qc_prediction_ukbb.csv")
df_prediction_pnc.to_csv("./results/qc_prediction_pnc.csv")
df_prediction_ppmi.to_csv("./results/qc_prediction_ppmi.csv")
df_prediction_adni.to_csv("./results/qc_prediction_adni.csv")
